In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

# ----------------------------
# 0. Atur path input & output
# ----------------------------
input_csv_path  = "./../dataset/solar_system_positions_with_velocity.csv"
output_csv_path = "./../dataset/orbit_lines_and_elements.csv"

# ----------------------------
# 1. Kepler helper functions
# ----------------------------
def solve_kepler_elliptic(M, e, tol=1e-9, max_iter=100):
    """
    Nyoba beberapa tebakan awal dan offset kecil kalau gagal konvergen.
    Jika tetap gagal, return M sebagai fallback.
    """
    def iterate(E):
        for _ in range(max_iter):
            f       = E - e*np.sin(E) - M
            f_prime = 1 - e*np.cos(E)
            delta   = -f/f_prime
            E      += delta
            if abs(delta) < tol:
                return E, True
        return E, False

    # Coba tebakan awal: M, M + e*sin(M), M + 0.001
    for E0 in (M, M + e*np.sin(M), M + 1e-3):
        E, ok = iterate(E0)
        if ok:
            return E

    # Kalau masih gagal:
    # Beri warning kecil dan return fallback
    print(f"Warning: Kepler solver gagal konvergen untuk M={M:.6f}, e={e:.6f}. Mengembalikan E=M.")
    return M  # fallback: E ≈ M

def rv_to_orbital_elements(r_vec, v_vec, mu, t0=0.0):
    # pastikan numeric array
    r_vec = np.asarray(r_vec, dtype=float)
    v_vec = np.asarray(v_vec, dtype=float)

    r     = np.linalg.norm(r_vec)
    v2    = np.dot(v_vec, v_vec)
    h_vec = np.cross(r_vec, v_vec);  h = np.linalg.norm(h_vec)
    K     = np.array([0.0,0.0,1.0], dtype=float)
    n_vec = np.cross(K, h_vec);      n = np.linalg.norm(n_vec)
    e_vec = ((v2 - mu/r)*r_vec - np.dot(r_vec,v_vec)*v_vec) / mu
    e     = np.linalg.norm(e_vec)

    energy = v2/2 - mu/r
    a      = np.inf if abs(e-1.0)<1e-8 else -mu/(2*energy)
    i      = np.arccos(h_vec[2]/h) if h>0 else 0.0

    if n>1e-8:
        raan = np.arccos(n_vec[0]/n)
        if n_vec[1]<0: raan = 2*np.pi - raan
    else:
        raan = 0.0

    if n>1e-8 and e>1e-8:
        argp = np.arccos(np.dot(n_vec,e_vec)/(n*e))
        if e_vec[2]<0: argp = 2*np.pi - argp
    else:
        argp = 0.0

    if e>1e-8:
        nu = np.arccos(np.dot(e_vec,r_vec)/(e*r))
        if np.dot(r_vec,v_vec)<0: nu = 2*np.pi - nu
    else:
        nu = 0.0

    if e<1.0:
        E0     = 2*np.arctan(np.sqrt((1-e)/(1+e))*np.tan(nu/2))
        M0     = (E0 - e*np.sin(E0)) % (2*np.pi)
        n_mean = np.sqrt(mu/a**3)
        T      = 2*np.pi/n_mean
    else:
        E0 = M0 = n_mean = T = None
    
    M0 = 0 if M0 is None or not np.isfinite(M0) else M0

    return {
        'a':    a,
        'e':    e,
        'i':    i,
        'raan': raan,
        'argp': argp,
        'M0':   M0,
        't0':   t0,
        'mu':   mu
    }

def elements_to_rv(el, t):
    a, e    = el['a'], el['e']
    i, raan, argp = el['i'], el['raan'], el['argp']
    M0, t0, mu   = el['M0'], el['t0'], el['mu']
    # hitung mean motion (untuk elliptic saja)
    n = np.sqrt(mu/a**3) if a and a != np.inf else 0.0

    dt = t - t0
    M  = (M0 + n*dt) % (2*np.pi)

    # solve Kepler
    E  = solve_kepler_elliptic(M, e)

    # posisi & kecepatan di perifocal
    x_p   = a*(np.cos(E)-e)
    y_p   = a*np.sqrt(1-e**2)*np.sin(E)
    E_dot = n/(1 - e*np.cos(E))
    vx_p  = -a*np.sin(E)*E_dot
    vy_p  =  a*np.sqrt(1-e**2)*np.cos(E)*E_dot

    # rotasi ke ECI
    cosO, sinO = np.cos(raan), np.sin(raan)
    cosw, sinw = np.cos(argp), np.sin(argp)
    cosi, sini = np.cos(i), np.sin(i)

    P = np.array([
        cosO*cosw - sinO*sinw*cosi,
        sinO*cosw + cosO*sinw*cosi,
        sinw*sini
    ], dtype=float)
    Q = np.array([
       -cosO*sinw - sinO*cosw*cosi,
       -sinO*sinw + cosO*cosw*cosi,
        cosw*sini
    ], dtype=float)

    r_eci = x_p*P + y_p*Q
    v_eci = vx_p*P + vy_p*Q
    return r_eci, v_eci

# ----------------------------
# 2. Main: baca, cast, hitung & propagasi
# ----------------------------
if __name__=='__main__':
    mu_sun = 1.32712440018e11  # km^3/s^2
    AU2km  = 1.496e+8
    day2s  = 86400.0

    # load & cast
    df = pd.read_csv(input_csv_path, parse_dates=['date'])
    cols = ['x_au','y_au','z_au','vx_au_per_day','vy_au_per_day','vz_au_per_day']
    df[cols] = df[cols].astype(float)

    elements_list = []
    lines = []

    for name, grp in df.groupby('name'):
        grp = grp.sort_values('date')

        # awal
        r0_au = grp.iloc[0][cols[:3]].values
        v0_au = grp.iloc[0][cols[3:]].values
        t0_dt = grp.iloc[0]['date']

        # akhir
        t1_dt = grp.iloc[-1]['date']

        # konversi
        r0 = r0_au * AU2km
        v0 = v0_au * (AU2km/day2s)
        t0 = t0_dt.timestamp()
        t1 = t1_dt.timestamp()

        # elemen
        el = rv_to_orbital_elements(r0, v0, mu_sun, t0)
        elements_list.append({
            'name': name,
            'sma':  el['a'],
            'ecc':  el['e'],
            'inc':  el['i'],
            'raan': el['raan'],
            'argp': el['argp'],
            'M0':   el['M0']
        })

        # garis orbit: 200 titik
        times = np.linspace(t0, t1, 200)
        for tt in times:
            r_new, _ = elements_to_rv(el, tt)
            dt = datetime.utcfromtimestamp(tt)
            lines.append({
                'name': name,
                'time': dt,
                'x_au': r_new[0]/AU2km,
                'y_au': r_new[1]/AU2km,
                'z_au': r_new[2]/AU2km
            })

    pd.DataFrame(elements_list) \
      .to_csv(output_csv_path.replace('.csv','_elements.csv'), index=False)
    pd.DataFrame(lines) \
      .to_csv(output_csv_path, index=False)

    print(">> Elemen orbital:", output_csv_path.replace('.csv','_elements.csv'))
    print(">> Garis orbit:", output_csv_path)


C:\Users\aroyy\AppData\Local\Temp\ipykernel_53244\2822168491.py:185: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(tt)
C:\Users\aroyy\AppData\Local\Temp\ipykernel_53244\2822168491.py:50: RuntimeWarning: divide by zero encountered in scalar divide
  e_vec = ((v2 - mu/r)*r_vec - np.dot(r_vec,v_vec)*v_vec) / mu
C:\Users\aroyy\AppData\Local\Temp\ipykernel_53244\2822168491.py:50: RuntimeWarning: invalid value encountered in multiply
  e_vec = ((v2 - mu/r)*r_vec - np.dot(r_vec,v_vec)*v_vec) / mu
C:\Users\aroyy\AppData\Local\Temp\ipykernel_53244\2822168491.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  energy = v2/2 - mu/r
C:\Users\aroyy\AppData\Local\Temp\ipykernel_53244\2822168491.py:185: DeprecationWarning: datetime.datetime.utcfromtimestamp() is depr

C:\Users\aroyy\AppData\Local\Temp\ipykernel_53244\2822168491.py:185: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(tt)
C:\Users\aroyy\AppData\Local\Temp\ipykernel_53244\2822168491.py:185: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(tt)
C:\Users\aroyy\AppData\Local\Temp\ipykernel_53244\2822168491.py:185: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.u

>> Elemen orbital: ./../dataset/orbit_lines_and_elements_elements.csv
>> Garis orbit: ./../dataset/orbit_lines_and_elements.csv


C:\Users\aroyy\AppData\Local\Temp\ipykernel_53244\2822168491.py:101: RuntimeWarning: invalid value encountered in sqrt
  n = np.sqrt(mu/a**3) if a and a != np.inf else 0.0
C:\Users\aroyy\AppData\Local\Temp\ipykernel_53244\2822168491.py:111: RuntimeWarning: invalid value encountered in sqrt
  y_p   = a*np.sqrt(1-e**2)*np.sin(E)
C:\Users\aroyy\AppData\Local\Temp\ipykernel_53244\2822168491.py:114: RuntimeWarning: invalid value encountered in sqrt
  vy_p  =  a*np.sqrt(1-e**2)*np.cos(E)*E_dot
C:\Users\aroyy\AppData\Local\Temp\ipykernel_53244\2822168491.py:185: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt = datetime.utcfromtimestamp(tt)
C:\Users\aroyy\AppData\Local\Temp\ipykernel_53244\2822168491.py:185: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal i